In [2]:
from google.colab import files
uploaded_1 = files.upload()
uploaded_2 = files.upload()

Saving articles.csv to articles.csv


Saving transactions-2p-42.csv to transactions-2p-42.csv


In [15]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("analysis").getOrCreate()

df_tr = spark.read.csv("transactions-2p-42.csv", header=True, inferSchema=True, sep=",")
df_art = spark.read.csv("articles.csv", header=True, inferSchema=True, sep=",")

In [17]:
# Ensure date type
df_tr = df_tr.withColumn("t_dat", F.to_date(F.col("t_dat"), "yyyy-MM-dd"))

# joining data
df = df_tr.join(df_art, on="article_id", how="left")

In [20]:
# 1.a. What are the product trend changes through seasons of the year

df = df.withColumn("month", F.month("t_dat")) \
       .withColumn("year", F.year("t_dat")) \
       .withColumn("quarter", F.quarter("t_dat")) \
       .withColumn("season",
                   F.when(F.col("month").isin(12,1,2),"Winter")
                    .when(F.col("month").isin(3,4,5),"Spring")
                    .when(F.col("month").isin(6,7,8),"Summer")
                    .otherwise("Autumn"))

season_stats = df.groupBy("season") \
    .agg(
        F.count("*").alias("transactions"),
        F.sum("price").alias("revenue"),
        F.countDistinct("customer_id").alias("unique_customers"),
        F.avg("price").alias("avg_price")
    ).orderBy("season")
season_stats.show()

+------+------------+------------------+----------------+--------------------+
|season|transactions|           revenue|unique_customers|           avg_price|
+------+------------+------------------+----------------+--------------------+
|Autumn|       79526| 2441.777796610195|           68668|0.030704144513872128|
|Spring|       85977| 2459.363118644167|           72586| 0.02860489571215752|
|Summer|       99222|2387.0828813560474|           82966| 0.02405800005398044|
|Winter|       71353| 1953.423474576357|           61617|0.027376893397283324|
+------+------------+------------------+----------------+--------------------+



In [22]:
# 1.b. What are the product trend changes through the years

year_stats = df.groupBy("year") \
    .agg(
        F.count("*").alias("transactions"),
        F.sum("price").alias("revenue"),
        F.countDistinct("customer_id").alias("unique_customers"),
        F.avg("price").alias("avg_price")
    ).orderBy("year")
year_stats.show()

+----+------------+-----------------+----------------+--------------------+
|year|transactions|          revenue|unique_customers|           avg_price|
+----+------------+-----------------+----------------+--------------------+
|2018|       88988|2644.698983050872|           75839|0.029719726064760102|
|2019|      247090|6596.948288136311|          180075| 0.02669856444265778|
+----+------------+-----------------+----------------+--------------------+



In [23]:
# 2.a. total revenue

total_revenue = df.agg(F.sum("price").alias("total_revenue"))
total_revenue.show()

+-----------------+
|    total_revenue|
+-----------------+
|9241.647271188613|
+-----------------+



In [24]:
# 2.b. number of customers

num_customers = df.select("customer_id").distinct().count()
num_customers

230688

In [25]:
# 2.c. number of transactions

num_transactions = df.count()
num_transactions

336078

In [26]:
# 2.d. average expenses per year per customer

df_year = df.withColumn("year", F.year("t_dat"))
cust_year = df_year.groupBy("customer_id", "year") \
    .agg(F.sum("price").alias("total_spending"))

# Average spending per customer per year
avg_expenses = cust_year.agg(F.avg("total_spending").alias("avg_expenses_per_year_per_customer"))
avg_expenses.show()

+----------------------------------+
|avg_expenses_per_year_per_customer|
+----------------------------------+
|               0.03611231613427958|
+----------------------------------+



In [27]:
# 3.a. Null values per column

null_report = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_report.show()

+----------+-----+-----------+-----+----------------+------------+---------+---------------+-----------------+------------------+-----------------------+-------------------------+-----------------+-----------------+-------------------------+---------------------------+--------------------------+----------------------------+-------------+---------------+----------+----------+--------------+----------------+----------+------------+----------------+------------------+-----------+-----+----+-------+------+
|article_id|t_dat|customer_id|price|sales_channel_id|product_code|prod_name|product_type_no|product_type_name|product_group_name|graphical_appearance_no|graphical_appearance_name|colour_group_code|colour_group_name|perceived_colour_value_id|perceived_colour_value_name|perceived_colour_master_id|perceived_colour_master_name|department_no|department_name|index_code|index_name|index_group_no|index_group_name|section_no|section_name|garment_group_no|garment_group_name|detail_desc|month|year|

In [28]:
# 3.b. Join mismatches

missing_articles = df.filter(F.col("prod_name").isNull()).count()
print("Missing article metadata:", missing_articles)

Missing article metadata: 0


In [29]:
# 3.c. Outlier detection – Price

price_stats = df.select(
    F.min("price").alias("min_price"),
    F.expr("percentile(price, 0.5)").alias("median"),
    F.expr("percentile(price, 0.99)").alias("p99"),
    F.max("price").alias("max_price")
)
price_stats.show()

+-----------------+------------------+------------------+------------------+
|        min_price|            median|               p99|         max_price|
+-----------------+------------------+------------------+------------------+
|1.864406779661E-4|0.0250338983050847|0.1016779661016949|0.5067796610169492|
+-----------------+------------------+------------------+------------------+



In [30]:
# 3.d. Duplicate transactions

duplicate_rows = df.groupBy(df.columns).count().filter("count > 1").count()
duplicate_rows

895

In [31]:
# 3.e. Date quality check

df.select(
    F.min("t_dat").alias("min_date"),
    F.max("t_dat").alias("max_date")
).show()

+----------+----------+
|  min_date|  max_date|
+----------+----------+
|2018-09-20|2019-09-19|
+----------+----------+



In [32]:
# 3.f. Invalid product attributes

df.groupBy("product_group_name").count().orderBy("count").show()

+-------------------+------+
| product_group_name| count|
+-------------------+------+
|   Interior textile|     1|
|Underwear/nightwear|     4|
|           Cosmetic|    20|
|              Items|    27|
|               Bags|    34|
|            Unknown|   382|
|          Nightwear|  3302|
|     Socks & Tights|  6925|
|              Shoes|  8465|
|        Accessories| 16781|
|          Underwear| 25114|
|           Swimwear| 28453|
|  Garment Full body| 35634|
| Garment Lower body| 75817|
| Garment Upper body|135119|
+-------------------+------+



In [33]:
# 4.a. Product groups with the highest customer loyalty

loyalty = df.groupBy("product_group_name", "customer_id") \
    .count() \
    .filter("count > 1") \
    .groupBy("product_group_name") \
    .count() \
    .orderBy(F.desc("count"))

loyalty.show()

+------------------+-----+
|product_group_name|count|
+------------------+-----+
|Garment Upper body|18350|
|Garment Lower body| 7750|
| Garment Full body| 2805|
|          Swimwear| 2323|
|         Underwear| 1551|
|       Accessories|  625|
|             Shoes|  334|
|    Socks & Tights|  191|
|         Nightwear|   63|
|           Unknown|    4|
+------------------+-----+



In [34]:
# 4.b. Most popular color category

df.groupBy("colour_group_name") \
  .count() \
  .orderBy(F.desc("count")) \
  .show()

+-----------------+------+
|colour_group_name| count|
+-----------------+------+
|            Black|116525|
|            White| 36624|
|        Dark Blue| 28495|
|      Light Beige| 11712|
|             Blue| 11686|
|       Light Blue| 10130|
|              Red|  9905|
|             Grey|  9396|
|       Light Pink|  8486|
|         Dark Red|  8483|
|        Off White|  8175|
|        Dark Grey|  7943|
|            Beige|  7687|
|   Greenish Khaki|  7657|
|       Dark Green|  6619|
|           Yellow|  4963|
|       Light Grey|  4602|
|             Pink|  4369|
|  Yellowish Brown|  3868|
|     Light Orange|  3854|
+-----------------+------+
only showing top 20 rows



In [35]:
# 4.c. Top 10 revenue-generating product types

top_product_types = df.groupBy("product_type_name") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")) \
    .limit(10)

top_product_types.show(truncate=False)

+-----------------+------------------+
|product_type_name|revenue           |
+-----------------+------------------+
|Trousers         |1584.8587627117877|
|Dress            |1188.2181694915025|
|Sweater          |867.5875762711836 |
|Blouse           |493.0956949152566 |
|Jacket           |418.0511525423692 |
|Skirt            |342.1530677966116 |
|T-shirt          |324.51540677966597|
|Top              |321.6338135593214 |
|Shorts           |312.98832203389884|
|Bra              |307.7484406779639 |
+-----------------+------------------+

